In [1]:
import datetime
import re
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from Project.Database import Db

In [2]:
year1_minutes, meta = Db.load_data(hourly=False, meta=True, year=1)
year2_minutes, meta = Db.load_data(hourly=False, meta=True, year=2)
year1_minutes

,Timestamp,DHW_ClothesWasherColdFlow,DHW_ClothesWasherHotFlow,DHW_DishwasherHotFlow,DHW_HeatPumpWaterHeaterEnergyTotal,DHW_HeatPumpWaterHeaterPowerTotal,DHW_ManifoldColdFlow,DHW_ManifoldHotFlow,DHW_MixValveColdFlow,DHW_RoomTempBasementHPWH,...,SHW_PSPSHW,SHW_SHWHX80galWaterFlow,Vent_HRVTempExhaustOut,Vent_HRVTempOutdoorin,Vent_HRVTempReturnIn,Vent_HRVTempSupplyOut,HVAC_HeatPumpIndoorUnitPower,HVAC_HeatPumpOutdoorUnitPower,Load_StatusClothesWasher,Load_StatusDryerPowerTotal
0,2013-07-01 00:01:14,0.0,0.0,0.00000,0.000000,6.920000,0.000000,0.00000,0.00000,21.238651,...,0.000056,0.00000,24.712040,23.144780,22.552117,23.758415,125.4000,922.6568,0,0
1,2013-07-01 00:02:14,0.0,0.0,0.00000,0.132000,6.920000,0.000000,0.00000,0.00000,21.223797,...,0.000027,0.00000,24.710517,23.143295,22.523656,23.737489,127.0400,919.2424,0,0
2,2013-07-01 00:03:14,0.0,0.0,0.00000,0.264000,6.920000,0.000000,0.00000,0.00000,21.263438,...,0.000028,0.00000,24.691049,23.148787,22.539345,23.742217,153.3050,920.8184,0,0
3,2013-07-01 00:04:14,0.0,0.0,0.00000,0.396000,6.920000,0.000000,0.00000,0.00000,21.228323,...,0.000040,0.00000,24.686121,23.120120,22.539554,23.727526,143.4575,922.3944,0,0
4,2013-07-01 00:05:14,0.0,0.0,0.00000,0.528000,6.920000,0.000000,0.00000,0.00000,21.246059,...,0.000011,0.00000,24.683265,23.179975,22.506425,23.705258,141.8150,925.2832,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518787,2014-06-30 23:29:22,0.0,0.0,1.86261,1074.687075,765.920563,11.297412,57.77676,8.69218,19.589094,...,11.326110,1867.29955,25.037362,25.339981,23.912489,24.423000,110.6250,1517.2800,0,0
518788,2014-06-30 23:30:22,0.0,0.0,1.86261,1087.432650,766.929871,11.297412,57.77676,8.69218,19.521485,...,11.916921,1867.29955,25.033280,25.318893,23.898320,24.403229,124.5800,1394.6256,0,0
518789,2014-06-30 23:31:22,0.0,0.0,1.86261,1100.200137,768.525940,11.297412,57.77676,8.69218,19.619391,...,11.259323,1867.29955,24.992671,25.261438,23.849174,24.392863,131.1450,1311.6304,0,0
518790,2014-06-30 23:59:22,0.0,0.0,1.86261,1173.107485,6.920000,11.297412,57.77676,8.69218,21.028008,...,12.130983,1867.29955,24.985092,25.130462,24.119352,24.579783,127.0400,1325.5504,0,0


In [3]:
#bath1 = meta.loc[(meta['Parameter'] == "Status_OnOff") & (meta['Measurement_Location'] == "Bath2") & (meta["Unnamed: 0"].str.contains('Status'))]['Unnamed: 0'].tolist()

In [4]:
# fig = px.scatter(year1_minutes, x="Timestamp", y=bath1)
# fig.show()

In [5]:
#Extract appliances from the meta data
appliance_condition = (lambda self: (
        (self["Parameter"] == "Status_OnOff") &
        (self["Subsystem"] == "Loads") &
        (~self["Unnamed: 0"].str.contains("Child")) &
        (~self["Unnamed: 0"].str.contains("Prnt"))
))

appliances_statusOnOff = meta.loc[appliance_condition]["Unnamed: 0"].tolist()

appliance_location = meta.loc[appliance_condition][["Unnamed: 0", "Measurement_Location"]].sort_values(by="Measurement_Location", ascending=True).rename(columns={"Unnamed: 0": "Appliance"}).set_index("Appliance")

appliance_location

KeyError: 'Unnamed: 0'

In [ ]:
#Combine minute data with the extracted appliances into a dataframe
minute_appliances_status = year1_minutes[["Timestamp"] + appliance_location.index.tolist()].copy()

In [ ]:
#Replaces values to be binary 0 or 1.
for i in appliances_statusOnOff:
    minute_appliances_status.loc[minute_appliances_status[i] < 0, i] = 0
    minute_appliances_status.loc[minute_appliances_status[i] > 0, i] = 1
minute_appliances_status

In [ ]:
appliance_job_list = []

for appliance in appliances_statusOnOff:
    name = " ".join(re.findall('[A-Z][^A-Z]*', appliance.split("Load_Status")[-1]))
    appliance_switch = minute_appliances_status[(minute_appliances_status[appliance] != minute_appliances_status[appliance].shift(1))][
                           ["Timestamp", appliance]][1:]

    df_indices = appliance_switch.index.tolist()
    for index, df_index in enumerate(df_indices):
        if appliance_switch.loc[df_index, appliance]:
            try:
                appliance_job_list.append({"appliance": name, "start": appliance_switch.loc[df_index, "Timestamp"], "end": appliance_switch.loc[df_indices[index+1], "Timestamp"], "location": appliance_location.loc[appliance, "Measurement_Location"]})
            except:
                continue

appliance_job_list

In [ ]:
fig = px.timeline(appliance_job_list, x_start="start", x_end="end", y="appliance", color="location")
fig.write_html(Db.get_save_file_directory("Appliance_status_gantt.html"))


In [ ]:
residens_condition = (lambda self: (
    (self["Subsystem"] == "Loads") &
    (self["Parameter"] == "Status_OnOff") &
    (self["Unnamed: 0"].str.contains("Sens"))
    )
)

residens_location = meta.loc[residens_condition, "Unnamed: 0"].tolist()
residens_location.sort()

# residens_location = meta.loc[residens_condition][["Unnamed: 0"]].rename(columns={"Unnamed: 0": "Appliance"}).set_index("Appliance")

residens_location

In [ ]:
#Combine minute data with the extracted residens into a dataframe
residens_location_status = year1_minutes[["Timestamp"] + residens_location].copy()

In [ ]:
#Replaces values to be binary 0 or 1.
for i in residens_location:
    residens_location_status.loc[residens_location_status[i] < 0, i] = 0
    residens_location_status.loc[residens_location_status[i] > 0, i] = 1
residens_location_status

In [ ]:
person_status_list = []

for col in residens_location:
    name = "".join(re.findall("[PC][a-z]+[AB]", col))
    person_location = residens_location_status[lambda self: (self[col] != self[col].shift(1))][
                           ["Timestamp", col]][1:]

    df_indices = person_location.index.tolist()
    for index, df_index in enumerate(df_indices):
        if person_location.loc[df_index, col]:
            try:
                person_status_list.append({"appliance": name, "start": person_location.loc[df_index, "Timestamp"], "end": person_location.loc[df_indices[index+1], "Timestamp"], "location": {"DOWN": "First floor", "UP": "Second floor"}["".join(col.split(name)[-1])]})
            except:
                continue
person_status_list

In [ ]:
fig = px.timeline(person_status_list, x_start="start", x_end="end", y="appliance", color="location")
fig.write_html(Db.get_save_file_directory("Person_status_gantt.html"))
# fig.show()

In [ ]:
fig = px.timeline(person_status_list+appliance_job_list, x_start="start", x_end="end", y="appliance", color="location")
# fig.show()

In [ ]:
fig.write_html(Db.get_save_file_directory("Person_appliance_status_gantt.html"))